In [10]:
pip install faiss

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
##VERSION1.0
import requests
from bs4 import BeautifulSoup
from ollama import Client

# Function to read web page content
def read_web_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Remove unnecessary elements
    for script in soup(["script", "style"]):
        script.extract()
    return soup.get_text()

# Function to answer questions using ollama
def answer_questions(content, question):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Here is some content: {content}\n\nBased on this content, please answer the following question: {question}\n\nAnswer:"}]
    response = client.chat(model='llama3.2', messages=chat_prompt)
    return response['message']['content']

def main():
    urls = input("Enter URLs (comma separated): ").split(',')
    question = input("Enter your question: ")
    combined_content = ' '.join([read_web_page(url.strip()) for url in urls])
    answer = answer_questions(combined_content, question)
    print(f"Answer: {answer}")

if __name__ == '__main__':
    main()

In [ ]:
##VERSION2.0
import requests
from bs4 import BeautifulSoup
from ollama import Client
import os

# Function to read web page content
def read_web_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        # Remove unnecessary elements
        for script in soup(["script", "style"]):
            script.extract()
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return ""

# Function to answer questions using ollama
def answer_questions(content, question):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Here is some content: {content}\n\nBased on this content, please answer the following question: {question}\n\nAnswer:"}]
    response = client.chat(model='llama3.2', messages=chat_prompt)
    return response['message']['content']

# Function to summarize content
def summarize_content(content):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Summarize the following content: {content}"}]
    response = client.chat(model='llama3.2', messages=chat_prompt)
    return response['message']['content']

def main():
    urls = input("Enter URLs (comma separated): ").split(',')
    combined_content = ' '.join([read_web_page(url.strip()) for url in urls])
    
    if not combined_content.strip():
        print("No valid content fetched from the provided URLs.")
        return
    
    summary = summarize_content(combined_content)
    print(f"Summary of the content: {summary}")
    
    with open("questions_and_answers.txt", "a") as file:
        while True:
            question = input("Enter your question: ")
            answer = answer_questions(combined_content, question)
            print(f"Answer: {answer}")
            
            file.write(f"Question: {question}\nAnswer: {answer}\n\n")
            
            continue_prompt = input("Do you want to ask another question? (yes/no): ").strip().lower()
            if continue_prompt != 'yes':
                break

if __name__ == '__main__':
    main()

In [9]:
import requests
from bs4 import BeautifulSoup
from ollama import Client
import os

# Function to read web page content
def read_web_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        # Remove unnecessary elements
        for script in soup(["script", "style"]):
            script.extract()
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return ""

# Function to answer questions using ollama
def answer_questions(content, question):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Here is some content: {content}\n\nBased on this content, please answer the following question: {question}\n\nAnswer:"}]
    response = client.chat(model='llama3.2', messages=chat_prompt)
    return response['message']['content']

# Function to summarize content
def summarize_content(content):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Summarize the following content: {content}"}]
    response = client.chat(model='llama3.2', messages=chat_prompt)
    return response['message']['content']

def main():
    urls = input("Enter URLs (comma separated): ").split(',')
    combined_content = ' '.join([read_web_page(url.strip()) for url in urls])
    
    if not combined_content.strip():
        print("No valid content fetched from the provided URLs.")
        return
    
    #summary = summarize_content(combined_content)
    #print(f"Summary of the content: {summary}")
    
    cache = {}
    
    with open("questions_and_answers.txt", "a") as file:
        while True:
            question = input("Enter your question: ")
            
            if question in cache:
                answer = cache[question]
            else:
                answer = answer_questions(combined_content, question)
                cache[question] = answer
            
            print(f"Answer: {answer}")
            
            file.write(f"Question: {question}\nAnswer: {answer}\n\n")
            
            continue_prompt = input("Do you want to ask another question? (yes/no): ").strip().lower()
            if continue_prompt != 'yes':
                break

if __name__ == '__main__':
    main()

Answer: No, the pop-up blocker in Microsoft Edge cannot block malware. According to the text, "If you still see pop-ups within a web page once this feature is turned on and you have tried the solutions listed above, they may be website advertisements created to look like pop-ups. Website advertisements cannot be blocked by the Edge pop-up blocker."
Answer: The content does not explicitly state that pop-ups are "good" or "bad". It only discusses how to block pop-ups in Microsoft Edge and explains that some pop-ups can be helpful (e.g., displaying a bank statement), while others can be distracting or malicious. However, it also notes that website advertisements created to look like pop-ups cannot be blocked by the Edge pop-up blocker, suggesting that not all pop-ups are created equal.
Answer: Yes, you can disable the pop-up blocker in Microsoft Edge. However, it's worth noting that disabling the pop-up blocker may expose your device to potential phishing scams or other malicious activiti

In [ ]:
#Improved caching
import requests
from bs4 import BeautifulSoup
from ollama import Client
import os
import json
import hashlib
from collections import OrderedDict
from datetime import datetime, timedelta

CACHE_FILE = "cache.json"
CACHE_EXPIRY_HOURS = 24

# Function to read web page content
def read_web_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        # Remove unnecessary elements
        for script in soup(["script", "style"]):
            script.extract()
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return ""

# Function to answer questions using ollama
def answer_questions(content, question):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Here is some content: {content}\n\nBased on this content, please answer the following question: {question}\n\nAnswer:"}]
    response = client.chat(model='llama3.2', messages=chat_prompt)
    return response['message']['content']

# Function to load cache from file
def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as file:
            return json.load(file, object_pairs_hook=OrderedDict)
    return OrderedDict()

# Function to save cache to file
def save_cache(cache):
    with open(CACHE_FILE, 'w') as file:
        json.dump(cache, file)

# Function to clean expired cache entries
def clean_cache(cache):
    now = datetime.now()
    keys_to_delete = [key for key, value in cache.items() if datetime.fromisoformat(value['timestamp']) < now - timedelta(hours=CACHE_EXPIRY_HOURS)]
    for key in keys_to_delete:
        del cache[key]

def main():
    urls = input("Enter URLs (comma separated): ").split(',')
    combined_content = ' '.join([read_web_page(url.strip()) for url in urls])
    
    if not combined_content.strip():
        print("No valid content fetched from the provided URLs.")
        return
    
    cache = load_cache()
    clean_cache(cache)
    
    content_hash = hashlib.md5(combined_content.encode()).hexdigest()
    
    with open("questions_and_answers.txt", "a") as file:
        while True:
            question = input("Enter your question: ")
            cache_key = f"{content_hash}_{question}"
            
            if cache_key in cache:
                answer = cache[cache_key]['answer']
            else:
                answer = answer_questions(combined_content, question)
                cache[cache_key] = {'answer': answer, 'timestamp': datetime.now().isoformat()}
                save_cache(cache)
            
            print(f"Answer: {answer}")
            
            file.write(f"Question: {question}\nAnswer: {answer}\n\n")
            
            continue_prompt = input("Do you want to ask another question? (yes/no): ").strip().lower()
            if continue_prompt != 'yes':
                break

if __name__ == '__main__':
    main()

In [ ]:
#vector db embedding
import requests
from bs4 import BeautifulSoup
from ollama import Client
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Function to read web page content
def read_web_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        # Remove unnecessary elements
        for script in soup(["script", "style"]):
            script.extract()
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return ""

# Function to answer questions using ollama
def answer_questions(content, question):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Here is some content: {content}\n\nBased on this content, please answer the following question: {question}\n\nAnswer:"}]
    response = client.chat(model='llama3.2', messages=chat_prompt)
    return response['message']['content']

# Function to generate embeddings
def generate_embeddings(texts, model):
    return model.encode(texts)

# Function to search for relevant content using FAISS
def search_faiss(index, query_embedding, texts, k=5):
    D, I = index.search(query_embedding, k)
    return [texts[i] for i in I[0]]

def main():
    urls = input("Enter URLs (comma separated): ").split(',')
    combined_content = ' '.join([read_web_page(url.strip()) for url in urls])
    
    if not combined_content.strip():
        print("No valid content fetched from the provided URLs.")
        return
    
    # Initialize the sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generate embeddings for the combined content
    content_sentences = combined_content.split('. ')
    content_embeddings = generate_embeddings(content_sentences, model)
    
    # Create a FAISS index and add the embeddings
    dimension = content_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(content_embeddings)
    
    with open("questions_and_answers.txt", "a") as file:
        while True:
            question = input("Enter your question: ")
            
            # Generate embedding for the question
            question_embedding = generate_embeddings([question], model)
            
            # Search for relevant content using FAISS
            relevant_content = search_faiss(index, question_embedding, content_sentences)
            combined_relevant_content = ' '.join(relevant_content)
            
            # Answer the question using the relevant content
            answer = answer_questions(combined_relevant_content, question)
            print(f"Answer: {answer}")
            
            file.write(f"Question: {question}\nAnswer: {answer}\n\n")
            
            continue_prompt = input("Do you want to ask another question? (yes/no): ").strip().lower()
            if continue_prompt != 'yes':
                break

if __name__ == '__main__':
    main()

In [ ]:
#prebuilding seperately
import requests
from bs4 import BeautifulSoup
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json

# Function to read web page content
def read_web_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        # Remove unnecessary elements
        for script in soup(["script", "style"]):
            script.extract()
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return ""

# Function to generate embeddings
def generate_embeddings(texts, model):
    return model.encode(texts)

def main():
    urls = input("Enter URLs (comma separated): ").split(',')
    combined_content = ' '.join([read_web_page(url.strip()) for url in urls])
    
    if not combined_content.strip():
        print("No valid content fetched from the provided URLs.")
        return
    
    # Initialize the sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generate embeddings for the combined content
    content_sentences = combined_content.split('. ')
    content_embeddings = generate_embeddings(content_sentences, model)
    
    # Create a FAISS index and add the embeddings
    dimension = content_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(content_embeddings)
    
    # Save the index and content sentences
    faiss.write_index(index, "vector_index.faiss")
    with open("content_sentences.json", "w") as f:
        json.dump(content_sentences, f)
    
    print("Vector database prebuilt and saved.")

if __name__ == '__main__':
    main()

In [ ]:
#loading and doing q and a
import requests
from bs4 import BeautifulSoup
from ollama import Client
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json

# Function to read web page content
def read_web_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        # Remove unnecessary elements
        for script in soup(["script", "style"]):
            script.extract()
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return ""

# Function to answer questions using ollama
def answer_questions(content, question):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Here is some content: {content}\n\nBased on this content, please answer the following question: {question}\n\nAnswer:"}]
    response = client.chat(model='llama3.2', messages=chat_prompt)
    return response['message']['content']

# Function to generate embeddings
def generate_embeddings(texts, model):
    return model.encode(texts)

# Function to search for relevant content using FAISS
def search_faiss(index, query_embedding, texts, k=5):
    D, I = index.search(query_embedding, k)
    return [texts[i] for i in I[0]]

def main():
    # Load the prebuilt FAISS index and content sentences
    index = faiss.read_index("vector_index.faiss")
    with open("content_sentences.json", "r") as f:
        content_sentences = json.load(f)
    
    # Initialize the sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    with open("questions_and_answers.txt", "a") as file:
        while True:
            question = input("Enter your question: ")
            
            # Generate embedding for the question
            question_embedding = generate_embeddings([question], model)
            
            # Search for relevant content using FAISS
            relevant_content = search_faiss(index, question_embedding, content_sentences)
            combined_relevant_content = ' '.join(relevant_content)
            
            # Answer the question using the relevant content
            answer = answer_questions(combined_relevant_content, question)
            print(f"Answer: {answer}")
            
            file.write(f"Question: {question}\nAnswer: {answer}\n\n")
            
            continue_prompt = input("Do you want to ask another question? (yes/no): ").strip().lower()
            if continue_prompt != 'yes':
                break

if __name__ == '__main__':
    main()

In [ ]:
#confluence
import requests
from bs4 import BeautifulSoup
from ollama import Client

# Function to read Confluence page content
def read_confluence_page(url, username, api_token):
    headers = {
        'Authorization': f'Basic {username}:{api_token}',
        'Content-Type': 'application/json'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    content = data['body']['storage']['value']
    soup = BeautifulSoup(content, 'html.parser')
    return soup.get_text()

# Function to answer questions using Llama
def answer_questions(content, question):
    client = Client(host='http://localhost:11434')
    chat_prompt = [{'role': 'user', 'content': f"Here is some content: {content}\n\nBased on this content, please answer the following question: {question}\n\nAnswer:"}]
    response = client.chat(model='llama3', messages=chat_prompt)
    return response['message']['content']

def main():
    urls = input("Enter Confluence URLs (comma separated): ").split(',')
    username = input("Enter your Confluence username: ")
    api_token = input("Enter your Confluence API token: ")
    
    combined_content = ' '.join([read_confluence_page(url.strip(), username, api_token) for url in urls])
    
    if not combined_content.strip():
        print("No valid content fetched from the provided URLs.")
        return
    
    while True:
        question = input("Enter your question: ")
        answer = answer_questions(combined_content, question)
        print(f"Answer: {answer}")
        
        continue_prompt = input("Do you want to ask another question? (yes/no): ").strip().lower()
        if continue_prompt != 'yes':
            break

if __name__ == '__main__':
    main()